In [1]:
from mixed_state_hmm import MixedHMM

# Define MHMM
mixed_states = [[0.0, 1.0, 0.5], [10.0, 1.0, 0.5], [20.0, 1.0, 0.5]]
mixed_states = [[0.0, 1.0, 0.2], [1.0, 1.0, 0.3], [2.0, 1.0, 0.4]]
mhmm = MixedHMM(mixed_states, trans_mat="random")


from oSLRAU_run import get_data
from spn.structure.leaves.parametric.Parametric import Gaussian, Bernoulli, Categorical,  In_Latent
from spn.structure.Base import Context
from spn.algorithms.LearningWrappers import learn_parametric
from spn.io.Graphics import plot_spn
import numpy as np
from spn.algorithms.Inference import log_likelihood
from sklearn.model_selection import train_test_split
from spn.algorithms.oSLRAU import oSLRAU, oSLRAUParams
from spn.algorithms.RSPN import RSPN
from spn.algorithms.TransformStructure import Prune ,Prune_oSLRAU

from hmmlearn import hmm
from spn.algorithms.Inference import log_likelihood


n_states = 3
n_dim = 2

num_time_steps_mix = 5
num_seq_mix = 20
rspn_data_mix= np.array([mhmm.sample_obs(num_time_steps_mix) for _ in range(num_seq_mix)])
rspn_data_mix.shape

rspn_data_mix = rspn_data_mix.reshape((rspn_data_mix.shape[0], -1))



num_variables = num_time_steps_mix * n_dim
num_latent_variables = n_states
num_latent_values = n_dim
unroll = 'backward'
full_update = False
update_leaves = True
len_sequence_varies = False
oSLRAU_params = oSLRAUParams(mergebatch_threshold=10, corrthresh=0.7, mvmaxscope=1, equalweight=True, currVals=True)


mini_batch_size = 2
update_after_no_min_batches = 2




def train_rspn_mix(train_data, test_data, rspn=None):

    if len(train_data.shape) == 3: train_data = np.squeeze(train_data)
    if len(test_data.shape) == 3: test_data = np.squeeze(test_data)

    if rspn is None:
    
        rspn = RSPN(num_variables=num_variables, num_latent_variables=num_latent_variables, num_latent_values=num_latent_values)
        first_mini_batch = train_data[0:mini_batch_size]
        n = first_mini_batch.shape[1]
        print(f"n: {n}") # = num_time_steps_mix * n_dim
        
        context = [Gaussian, Categorical]*(int(n/2)) # Bernoulli Categorical
        ds_context = Context(parametric_types=context).add_domains(first_mini_batch[:, :num_variables])
        spn, initial_template_spn, top_spn = rspn.build_initial_template(first_mini_batch, ds_context, len_sequence_varies)

    
    no_of_minibatches = int(train_data.shape[0] / mini_batch_size)

    print(f"no of minibatches: {no_of_minibatches}")

    
    for i in range(1, no_of_minibatches):
        mini_batch = train_data[i * mini_batch_size: (i+1) * mini_batch_size]
    
        update_template = False
        
        if i % update_after_no_min_batches == 0:
            print(i)
            update_template = True
    
        template_spn = rspn.learn_rspn(mini_batch, update_template, oSLRAU_params, unroll, full_update, update_leaves,
                                       len_sequence_varies)
           
            
    test_ll = np.mean(rspn.log_likelihood(test_data, unroll, len_sequence_varies))


    return test_ll, rspn, template_spn



def avg_ll_mix(data, num_epochs=1, do_plot_spn=True):
    ll_list = []
    rspn=None
    train_data, val_data= train_test_split(data, test_size=0.1, random_state=42)
    for i in range(num_epochs):
        print(train_data.shape)
        
        
        epoch_ll, rspn, template_spn = train_rspn_mix(train_data, val_data, rspn)        
        print(f"epoch_ll: {epoch_ll}")
        
        ll_list.append(epoch_ll)

    average_ll = np.mean(np.array(ll_list))
    print(f" \n\n\n average ll: {average_ll}\n\n\n")

    if do_plot_spn:
        plot_spn(template_spn, 'rspn_final_template.pdf')
    
    return rspn



rspn_mix = avg_ll_mix(rspn_data_mix, do_plot_spn=False)

test_len_mix = 5
test_data_mix = np.array([mhmm.sample_obs(num_time_steps_mix) for _ in range(test_len_mix)])
avg_ll_mix_mhmm = mhmm.true_avg_ll(test_data_mix)
print(f"==>> avg_ll_mix_mhmm: {avg_ll_mix_mhmm}")


if len(test_data_mix.shape) == 3: test_data_mix = np.squeeze(test_data_mix)
print(test_data_mix.shape)
test_data_mix = test_data_mix.reshape((len(test_data_mix), -1))

rspn_test_ll_mix = np.mean(rspn_mix.log_likelihood(test_data_mix, unroll, len_sequence_varies))
print(f"==>> rspn_test_ll_mix: {rspn_test_ll_mix}")


==>> rspn_data_mix.shape: (20, 10)
(18, 10)
n: 10
==>> data.shape: (2, 10)
len(self.meta_types): 10
self.meta_types.values(): dict_values([<MetaType.REAL: 1>, <MetaType.DISCRETE: 3>, <MetaType.REAL: 1>, <MetaType.DISCRETE: 3>, <MetaType.REAL: 1>, <MetaType.DISCRETE: 3>, <MetaType.REAL: 1>, <MetaType.DISCRETE: 3>, <MetaType.REAL: 1>, <MetaType.DISCRETE: 3>])
len(self.meta_types.values()): 10
data[:, col]: [0.27364079 0.8442725 ]
feature_meta_type: MetaType.REAL
==>> domain_values: [0.27364078792507585, 0.8442725036276784]
len(self.meta_types.values()): 10
data[:, col]: [0. 0.]
feature_meta_type: MetaType.DISCRETE
len(self.meta_types.values()): 10
data[:, col]: [ 2.08677443 -0.83044135]
feature_meta_type: MetaType.REAL
==>> domain_values: [-0.8304413493423204, 2.086774428382906]
len(self.meta_types.values()): 10
data[:, col]: [0. 0.]
feature_meta_type: MetaType.DISCRETE
len(self.meta_types.values()): 10
data[:, col]: [ 4.26401376 -0.12405932]
feature_meta_type: MetaType.REAL
==>> domain_

/Users/dariyankhan/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Work (one drive)/Imperial/MIT/exchange_year/summer-UROP-knowledge-distillation/policy_stuff/policy-learn/policy-venv_new/lib/python3.10/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/dariyankhan/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Work (one drive)/Imperial/MIT/exchange_year/summer-UROP-knowledge-distillation/policy_stuff/policy-learn/oSLRAU_and_RSPN/mixed_state_hmm.py:124: RuntimeWarning: divide by zero encountered in log
  traj_ll += np.log(self.init_probs[traj[0]])


==>> avg_ll_mix_mhmm: -11.080123968350788
(5, 5, 2)
Length of the sequence in mini_batch: 1
==>> data.shape: (5, 10)
==>> self.num_variables: 10
==>> self.len_sequence: 1
Evaluating rspn bottom up
==>> rspn_test_ll_mix: -11.450159189472775
